In [142]:
%run Preprocessing.ipynb
import pandas as pd
import numpy as np
import nbimporter
import feature_engineering
from datetime import datetime, timedelta
from functools import reduce
import glob
import os
import warnings
print("done")
#%run Preprocessing.ipynb

warnings.simplefilter(action='ignore', category=FutureWarning)

done


# 1. Preprocessing

## 1.1 Climate Data

### 1.1.1 Climate Data - Import and downsampling to 15min steps

In [143]:
climate_data = ["Temperature", "Precipitation"]
station_names = ["Berlin", "BW", "NRW"]
temp_path =rf"C:\Users\Isabell\EMSI\emsi_ml\Wetter\{station}\{data}"
file_name = "produkt*"
date_column = "MESS_DATUM"
date_format = '%Y%m%d%H%M'

station_climate_list = []

for data in climate_data:
    for station in station_names:
        temperature_dic = importing_data(temp_path, file_name, date_column, date_format)
        for key, climate_df in temperature_dic.items():

            exclude_col = "DateTime"
            climate_df.columns = [station + col if col != exclude_col else col for col in climate_df.columns]
            nan_count = climate_df.isna().sum()
            print(nan_count)
        #print(temperature_dic)
        
        station_climate_list.append(temperature_dic)

#print(temperature_dic.keys())
print("done")

BerlinSTATIONS_ID    0
BerlinMESS_DATUM     0
Berlin  QN           0
BerlinPP_10          0
BerlinTT_10          0
BerlinTM5_10         0
BerlinRF_10          0
BerlinTD_10          0
Berlineor            0
DateTime             0
dtype: int64
BerlinSTATIONS_ID    0
BerlinMESS_DATUM     0
Berlin  QN           0
BerlinPP_10          0
BerlinTT_10          0
BerlinTM5_10         0
BerlinRF_10          0
BerlinTD_10          0
Berlineor            0
DateTime             0
dtype: int64
BerlinSTATIONS_ID    0
BerlinMESS_DATUM     0
Berlin  QN           0
BerlinPP_10          0
BerlinTT_10          0
BerlinTM5_10         0
BerlinRF_10          0
BerlinTD_10          0
Berlineor            0
DateTime             0
dtype: int64
BerlinSTATIONS_ID    0
BerlinMESS_DATUM     0
Berlin  QN           0
BerlinPP_10          0
BerlinTT_10          0
BerlinTM5_10         0
BerlinRF_10          0
BerlinTD_10          0
Berlineor            0
DateTime             0
dtype: int64
BerlinSTATIONS_ID    0
Berli

In [150]:
climate_resampled = []
for station in station_climate_list:
    resampled_dic = {}
    for key, climate_df in station.items():
        print("exec")
        #climate_df.reset_index(drop=True)
        try:
            climate_df.set_index("DateTime", inplace=True)
        except:
            pass
            #print(climate_df)
            
        numeric_cols = climate_df.select_dtypes(include='number').columns
        df_resampled_numeric = climate_df[numeric_cols].resample('15T').mean()
    
        # Resample non-numeric columns
        non_numeric_cols = climate_df.select_dtypes(exclude='number').columns
        df_resampled_non_numeric = climate_df[non_numeric_cols].resample('15T').agg(lambda x: x.mode()[0] if not x.mode().empty else x.iloc[0])
    
        # Combine the resampled numeric and non-numeric dataframes
        df_resampled = pd.concat([df_resampled_numeric, df_resampled_non_numeric], axis=1)
    
        resampled_dic[key] = df_resampled
    climate_resampled.append(resampled_dic)
        #print(temperature_dic[key])
print("done")
print(len(climate_resampled))
print(climate_resampled[0])

exec
exec
exec
exec
exec
exec
exec
exec
exec
exec
exec
exec
exec
exec
exec
exec
exec
exec
exec
exec
exec
exec
exec
exec
exec
exec
exec
exec
exec
exec
done
6
{2012:                      BerlinSTATIONS_ID  BerlinMESS_DATUM  Berlin  QN  \
DateTime                                                               
2012-01-01 00:00:00              433.0      2.012010e+11         3.0   
2012-01-01 00:15:00              433.0      2.012010e+11         3.0   
2012-01-01 00:30:00              433.0      2.012010e+11         3.0   
2012-01-01 00:45:00              433.0      2.012010e+11         3.0   
2012-01-01 01:00:00              433.0      2.012010e+11         3.0   
...                                ...               ...         ...   
2012-12-31 22:45:00              433.0      2.012123e+11         3.0   
2012-12-31 23:00:00              433.0      2.012123e+11         3.0   
2012-12-31 23:15:00              433.0      2.012123e+11         3.0   
2012-12-31 23:30:00              433.0      

## 1.2 Importing Data - Football

In [114]:

temp_path =r"Spielplan"
file_name = "*.csv"
date_column = "DateTime"
date_format = '%d. %b %y %H:%M'
other_date_format = '%d.%m.%Y %H:%M'

help(importing_data)

football_dic = importing_data(temp_path, file_name, date_column, date_format, other_date_format, football_dataset = True)


    

Help on function importing_data in module __main__:

importing_data(temp_path, temp_file, column_name, *date_format, football_dataset=False)



## Adding new columns for Matches at same time

In [115]:

for key, match_set in football_dic.items():
    match_set = match_set.loc[:, ["Land1", "Land2", "Runde", "Liga", "DateTime"]]
    #print(spielplan)
    grouped = match_set.groupby("DateTime")
    results = pd.DataFrame()

    for name, group in grouped:
        if len(group) == 1:
            results = pd.concat([results, group])
        else:
            land1_value = group.iloc[0]['Land1']
            land2_value = group.iloc[0]['Land2']

            new_row = {
                "DateTime" : name,
                'Land1': land1_value,
                'Land2': land2_value,
                "Runde" : group.iloc[1]["Runde"],
                "Liga": group.iloc[1]["Liga"],
                'Land3': group.iloc[1]['Land1'],
                'Land4': group.iloc[1]['Land2'],
                
            }

            new_df = pd.DataFrame([new_row])

            results = pd.concat([results, new_df], ignore_index = True)
    results.set_index("DateTime")
    football_dic[key] = results

print(football_dic.keys())

dict_keys([2021, 2016, 2012, 2014, 2018])


## 1.3 Importing electricty data

In [155]:
temp_path =r"Stromverbrauch\Viertel"
file_name = "*.csv"
date_column = "Datum von"
date_format = '%d.%m.%Y %H:%M'

elec_dic = importing_data(temp_path, file_name, date_column, date_format)
for key, df in elec_dic.items():
    zero_count = df.apply(lambda col: (col == 0).sum())
    print(zero_count)
#print(elec_dic.keys())

Datum von                                      0
Datum bis                                      0
Gesamt (Netzlast) [MWh] Originalauflösungen    0
DateTime                                       0
dtype: int64
Datum von                                      0
Datum bis                                      0
Gesamt (Netzlast) [MWh] Originalauflösungen    0
DateTime                                       0
dtype: int64
Datum von                                      0
Datum bis                                      0
Gesamt (Netzlast) [MWh] Originalauflösungen    0
DateTime                                       0
dtype: int64


In [75]:

#index_counter = 0
counter_limit = len(spielplan_list) - 1

merged_df_dic = {}

print(dfs_by_year.keys())

for key,value in dfs_by_year.items():
    print(key)
    found_item = False
    index_counter = 0
    while found_item == False:
        spielplan = spielplan_list[index_counter]
        #rint(spielplan)
        spielplan_name_list = spielplan.columns.tolist()

        if key == spielplan["DateTime"].iloc[0].year:
            merged_df = pd.merge(value, spielplan, left_on="Datum von", right_on='DateTime', how='outer')
            
            for index, row in spielplan.iterrows():
                start_time = row["DateTime"]
                
                until_time = row["DateTime"] + timedelta(minutes=105)
                #print(until_time)
                fill_condition = (merged_df['Datum von']>= start_time) & (merged_df['Datum von'] <= until_time)
                merged_df.loc[fill_condition,spielplan_name_list] = merged_df.loc[fill_condition, spielplan_name_list].fillna(method='ffill')
                print(merged_df)
            if not merged_df.empty:
                #print(year)
                merged_df_dic[key] = merged_df
            #merged_df_list.append(merged_df)
            found_item = True
        elif index_counter == counter_limit:
            found_item = True
        else:
            index_counter += 1
    


dict_keys([2015, 2016, 2017, 2018, 2019, 2021, 2023])
2015
2016
               Datum von         Datum bis  \
0    2016-06-10 00:00:00  10.06.2016 00:15   
1    2016-06-10 00:15:00  10.06.2016 00:30   
2    2016-06-10 00:30:00  10.06.2016 00:45   
3    2016-06-10 00:45:00  10.06.2016 01:00   
4    2016-06-10 01:00:00  10.06.2016 01:15   
...                  ...               ...   
2971 2016-07-10 22:45:00  10.07.2016 23:00   
2972 2016-07-10 23:00:00  10.07.2016 23:15   
2973 2016-07-10 23:15:00  10.07.2016 23:30   
2974 2016-07-10 23:30:00  10.07.2016 23:45   
2975 2016-07-10 23:45:00  11.07.2016 00:00   

     Gesamt (Netzlast) [MWh] Originalauflösungen Land1 Land2 Runde Liga  \
0                                      11.930,50   NaN   NaN   NaN  NaN   
1                                      11.764,50   NaN   NaN   NaN  NaN   
2                                      11.602,50   NaN   NaN   NaN  NaN   
3                                      11.495,00   NaN   NaN   NaN  NaN   
4       

In [78]:
merged_dic_final = {}
for key,value in merged_df_dic.items():
    #value.drop(columns = ["level_0"], inplace = True)
    temp_df = temperature_dic[key]
    temp_df.reset_index(inplace = True)
    #print(temp_df)
    temp_df.rename(columns={"MESS_DATUM": "DateTime"}, inplace = True)
    #print(value)
    temp_df.set_index("DateTime", inplace=True)

    numeric_cols = temp_df.select_dtypes(include='number').columns
    df_resampled_numeric = temp_df[numeric_cols].resample('15T').mean()

    # Resample non-numeric columns
    non_numeric_cols = temp_df.select_dtypes(exclude='number').columns
    df_resampled_non_numeric = temp_df[non_numeric_cols].resample('15T').agg(lambda x: x.mode()[0] if not x.mode().empty else x.iloc[0])

    # Combine the resampled numeric and non-numeric dataframes
    df_resampled = pd.concat([df_resampled_numeric, df_resampled_non_numeric], axis=1)
    
    #temp_df_resampled = temp_df.resample("15T").mean()

    df_resampled.reset_index(inplace = True)
    value.reset_index(inplace = True)

    merged_df_final = pd.merge(value, df_resampled, left_on="Datum von", right_on='DateTime', how = "left")

    if not df_resampled.empty:
        merged_dic_final[key] = merged_df_final
        
print(merged_dic_final.keys())

C:\Users\Isabell\AppData\Local\Temp\ipykernel_1476\3495137154.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df.rename(columns={"MESS_DATUM": "DateTime"}, inplace = True)


dict_keys([2016, 2018, 2021])


## 1.4 Merging Dataframes

In [152]:
dfs = []
for key, elec in elec_dic.items():
    dfs_year = []
    dfs_year.append(pd.DataFrame(elec))
    dfs_year.append(pd.DataFrame(football_dic[key]))
    for station in climate_resampled:
        dfs_year.append(pd.DataFrame(station[key]))
        #print(station[key]["DateTime"])
    merged_df = reduce(lambda left, right: pd.merge(left, right, on='DateTime', how='left'), dfs_year)

    print(merged_df)

           Datum von         Datum bis  \
0   10.07.2016 00:00  10.07.2016 00:15   
1   10.07.2016 00:15  10.07.2016 00:30   
2   10.07.2016 00:30  10.07.2016 00:45   
3   10.07.2016 00:45  10.07.2016 01:00   
4   10.07.2016 01:00  10.07.2016 01:15   
..               ...               ...   
91  10.07.2016 22:45  10.07.2016 23:00   
92  10.07.2016 23:00  10.07.2016 23:15   
93  10.07.2016 23:15  10.07.2016 23:30   
94  10.07.2016 23:30  10.07.2016 23:45   
95  10.07.2016 23:45  11.07.2016 00:00   

   Gesamt (Netzlast) [MWh] Originalauflösungen            DateTime Land1  \
0                                    10.603,75 2016-07-10 00:00:00   NaN   
1                                    10.444,50 2016-07-10 00:15:00   NaN   
2                                    10.283,50 2016-07-10 00:30:00   NaN   
3                                    10.237,75 2016-07-10 00:45:00   NaN   
4                                    10.018,50 2016-07-10 01:00:00   NaN   
..                                     